<a href="https://colab.research.google.com/github/varunjain09/Projects/blob/main/Data_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install newspaper3k
!pip install newspaper3k sentence-transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 68.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.7 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=dbdabd2bdb8edbcfe42570fca1f6fcaffc5ba10e5dd174ac623bc77a28752fb0
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=9a384c9fc069f5f72c5ad3f6bed38dd007fa931a7a3dc89a540b89e657fe4a61
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1e

In [3]:
!pip install 'lxml[html_clean]'

In [4]:
!pip install sentence-transformers faiss-cpu transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.5 MB/s eta 0:00:00


In [7]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [8]:
import os
import numpy as np
from newspaper import Article
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

#  Load Mistral LLM
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

rag_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

#  Embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Step 1: Scrape article
def extract_article_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        print(f"❌ Error with {url}: {e}")
        return ""

# Step 2: Embed text into chunks
def embed_text(text, chunk_size=512):
    if not text:
        return [], []
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    embeddings = embedder.encode(chunks, convert_to_tensor=False)
    return chunks, embeddings

#  Step 3: FAISS index
def build_faiss_index(embedding_batches):
    flat_embeddings = [e for batch in embedding_batches for e in batch]
    if not flat_embeddings:
        raise ValueError("No embeddings to index.")
    matrix = np.vstack(flat_embeddings).astype("float32")
    index = faiss.IndexFlatL2(matrix.shape[1])
    index.add(matrix)
    return index

# Step 4: Semantic search
def search_faiss(index, query, chunks, k=3):
    query_vec = embedder.encode([query]).astype("float32")
    D, I = index.search(query_vec, k)
    return [chunks[i] for i in I[0]]

#  Step 5: RAG Query
def answer_query(query, index, chunks):
    context = "\n\n".join(search_faiss(index, query, chunks))
    prompt = f"[INST] Use the context below to answer the question:\n\n{context}\n\nQuestion: {query} [/INST]"

    output = rag_pipeline(prompt, max_new_tokens=300, do_sample=True, temperature=0.7)
    return output[0]['generated_text'].replace(prompt, "").strip()


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
t

In [ ]:
if __name__ == "__main__":
    urls = [
        "https://en.wikipedia.org/wiki/Retrieval-augmented_generation",
        "https://blog.google/technology/ai/google-gemini-ai/"
    ]

    all_chunks = []
    all_embeddings = []

    for url in urls:
        print(f"🔗 Scraping: {url}")
        text = extract_article_text(url)
        chunks, embeddings = embed_text(text)
        if chunks and len(embeddings) > 0:
           all_chunks.extend(chunks)
           all_embeddings.append(embeddings)

    print("✅ Building FAISS index...")
    index = build_faiss_index(all_embeddings)
    print(f"✅ FAISS index built with {index.ntotal} vectors.")

    while True:
        query = input("\nAsk a question (or type 'exit'): ")
        if query.lower() == "exit":
            break
        answer = answer_query(query, index, all_chunks)
        print("\n Answer:\n", answer)


🔗 Scraping: https://en.wikipedia.org/wiki/Retrieval-augmented_generation
🔗 Scraping: https://blog.google/technology/ai/google-gemini-ai/
✅ Building FAISS index...
✅ FAISS index built with 31 vectors.

Ask a question (or type 'exit'): what is RAG?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 Answer:
 Retrieval-augmented generation (RAG) is a technique that combines external documents and user input into an LLM prompt to generate tailored output. It involves retrieving relevant documents based on user queries using LLM embeddings and storing them in a vector database for efficient retrieval. RAG can improve the accuracy and relevance of LLM responses by providing source references that enable users to verify information.
